In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from compare_tools.train_utils import judgment_labels
from htrc_features import utils
import duckdb
import dask.dataframe as dd

In [3]:
con = duckdb.connect('/data/saddl/full-fixed2.duckdb', read_only=False, config={'max_memory':'16G', 'threads':4})

## Add Metadata

This code sometimes fails, sometimes succeeds. DuckDB is new!

In [6]:
con.execute("DROP TABLE IF EXISTS meta")
con.execute("CREATE TABLE meta AS SELECT * FROM parquet_scan('/data/saddl/meta.parquet/*.parquet') ORDER BY htid").fetch_df()

,Count
0,8741223


In [7]:
con.execute("SELECT * FROM meta LIMIT 1").fetch_df()

,htid,access,rights,ht_bib_key,description,source,source_bib_num,oclc_num,isbn,issn,...,pub_place,lang,bib_fmt,collection_code,content_provider_code,responsible_entity_code,digitization_agent_code,access_profile_code,author,page_count
0,aeu.ark:/13960/t0000bj8z,allow,pd,100246665,NaN,AEU,4935572,716089847,0665003617,NaN,...,onc,eng,BK,AEU,ualberta,ualberta,ia,open,"Burwash, N. 1839-1918.",108


## Create Predictions Table

In [4]:
con.execute("CREATE TABLE predictions(target VARCHAR, candidate VARCHAR, "
            "SWSM FLOAT, SWDE FLOAT, WP_DV FLOAT, PARTOF FLOAT, CONTAINS FLOAT, "
            "\"OVERLAPS\" FLOAT, AUTHOR FLOAT, SIMDIFF FLOAT, GRSIM FLOAT, RANDDIFF FLOAT, "
            "guess VARCHAR, relatedness FLOAT, flipped BOOLEAN)").fetchall()

[]

### Insert all dataset files

In [5]:
from compare_tools.utils import load_with_target, add_stubs, flip_judgments
# this is the inventory created by compare_tools/scripts/inventory-data.sh
with open('/tmp/all_predictions_data.txt', mode='r') as f:
    processed = pd.DataFrame(f.read().split('\n'), columns=['path'])
    processed = processed[~(processed.path == '')]
processed['clean_htid'] = processed.path.apply(lambda x: os.path.split(x)[-1])
processed['htid'] = processed.clean_htid.apply(utils.extract_htid)

In [6]:
# Ignore predictions that are already in the database
in_db1 = set(con.execute("SELECT DISTINCT target FROM predictions WHERE flipped == False").fetch_df().target)
in_db2 = set(con.execute("SELECT DISTINCT candidate FROM predictions WHERE flipped == True").fetch_df().candidate)
in_db = in_db1.intersection(in_db2)
processed = processed[~processed.htid.isin(in_db)]

len(processed), len(in_db)

(1179160, 0)

In [ ]:
%%time
# If you're trying to insert stuff after creating an index: 1) Why? 2) Delete the index and rebuild later.
con.execute("DROP INDEX IF EXISTS target_idx;")
con.execute("DROP INDEX IF EXISTS candidate_idx;")

for i, path in enumerate(processed.path + '.predictions.parquet'):
    if i % 1000 == 0:
        print(i, end=', ')
    df = load_with_target(path)
    df = flip_judgments(df)
    rel = con.from_df(df)
    rel.insert_into('predictions')

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 

Backup table (as multiple parquet files, using ranges):

In [15]:
%%time
unique_targets = con.execute("SELECT DISTINCT target FROM predictions ORDER BY target").fetch_df()
ranges = ['000'] + unique_targets.iloc[::100000].target.tolist() + ['zzz']
ranges = list(zip(ranges[:-1], ranges[1:]))

for i, (start, end) in enumerate(ranges):
    #clean_start = utils.clean_htid(start) if '.' in start else start
    #clean_end = utils.clean_htid(end) if '.' in end else end
    outpath = f"/tmp/predictions-backup-sorted.{i:03.0f}.parquet"
    print(outpath, start, end)
    con.execute(f"COPY (SELECT * FROM PREDICTIONS WHERE target > '{start}' AND target <= '{end}' ORDER by target) to '{outpath}' (FORMAT 'parquet')").fetch_df()

000 02870z575
02870z575 chi.79231875
chi.79231875 coo.31924055523389
coo.31924055523389 coo.31924082837430
coo.31924082837430 coo1.ark:/13960/t7rn3r264
coo1.ark:/13960/t7rn3r264 hvd.32044055324412
hvd.32044055324412 hvd.32044105340848
hvd.32044105340848 hvd.hn5d62
hvd.hn5d62 hvd.hxhszq
hvd.hxhszq ien.35556030579825


KeyboardInterrupt: 

In [14]:
!ls -lh /data/saddl/full-fixed.duckdb

-rw-r--r--. 1 peter.organisciak domain users 32G Jul  8 14:08 /data/saddl/full-fixed.duckdb


In [13]:
!ls -lh /tmp/predictions-backup.parquet

-rw-r--r--. 1 peter.organisciak domain users 8.5G Jul  8 12:13 /tmp/predictions-backup.parquet


### Create indexes

Indexes aren't persisted (https://duckdb.org/docs/sql/indexes), so create them every time you're looking to query the table.

In [8]:
%%time
con.execute("CREATE INDEX htid_idx ON meta (htid);")
con.execute("CREATE INDEX target_idx ON predictions (target);")
con.execute("CREATE INDEX candidate_idx ON predictions (candidate);").fetch_df()

CPU times: user 3min 16s, sys: 12 s, total: 3min 28s
Wall time: 3min 28s


,Count


### Create a table of the reciprocal relationships

The `predictions` table includes multiple book references, because it may analyze `target:book1/candidate:book2` as well as `target:book2/candidate:book1`. Those were sorted before inputting into `predictions`, so `target<candidate` alphabetically. `clean_predictions` subsequently takes the mean of also book1<->book2 inferences and saves the info.

A view is useful, but for profiling I'll save to parquet, then create a new table and a new sorted table. CREATE TABLE AS uses too much memory - which is why I CREATE VIEW, then save to parquet, then CREATE TABLE from the parquet.

Field names in the `SELECT` clause are quoted simply because one field `OVERLAPS` - is reserved.

In [32]:
%%time
## DO YOU HAVE ENOUGH MEMORY? Probably not... in which case it's better to process from the chunked parquet backups
enough_memory = False
include_std = False # Get the standard deviation of all reciprocal pairs, per label, and return a mean stddev

if enough_memory:
    # AGGREGATE reciprocal data
    select_cols = ", ".join(['target', 'candidate'] + [f"AVG(\"{x.lower()}\") as \"{x}\"" for x in judgment_labels+['relatedness']])
    if include_std:
        select_cols += ", " + "+".join([f"STDDEV_POP(\"{x.lower()}\")" for x in judgment_labels]) + 'as STD_DIFF'
    q = "CREATE VIEW clean_predictions_view AS SELECT " + select_cols + ", COUNT(1) AS count FROM predictions GROUP BY target, candidate HAVING count > 1"
    print(q)
    
    # create view, because we'll be returning to this
    con.execute(q)
    df = con.execute("SELECT * FROM clean_predictions_view LIMIT 1").fetch_df()
    con.execute("COPY (SELECT * FROM clean_predictions_view) TO '/tmp/clean_predictions.parquet' (FORMAT 'parquet')")
    con.execute("DROP TABLE IF EXISTS clean_predictions")
    con.execute("CREATE TABLE clean_predictions(target VARCHAR, candidate VARCHAR, "
                "SWSM FLOAT, SWDE FLOAT, WP_DV FLOAT, PARTOF FLOAT, CONTAINS FLOAT, "
                "\"OVERLAPS\" FLOAT, AUTHOR FLOAT, SIMDIFF FLOAT, GRSIM FLOAT, RANDDIFF FLOAT, "
                "relatedness FLOAT, count INT)")
    con.execute("INSERT INTO clean_predictions SELECT * FROM parquet_scan('/tmp/clean_predictions.parquet')")

else:
    # This is what I'm doing as I scale up
    # I gradually pull in the parquet files, which had been saved in alphabetically order (by target), and GROUP BY target, candidate in order
    # to average the softmax probabilities. This 'clean_predictions' table will be the final one that I use for everything.
    # Currently, I don't have the table doubled up with target and candidate (and partof/contains) swapped - adding that and sorting again on target
    # may be the next step.
    import glob
    ppaths = sorted(glob.glob('/tmp/predictions-backup-sorted.*.parquet'))
    
    con.execute("DROP TABLE IF EXISTS clean_predictions")
    con.execute("CREATE TABLE clean_predictions(target VARCHAR, candidate VARCHAR, "
                "SWSM FLOAT, SWDE FLOAT, WP_DV FLOAT, PARTOF FLOAT, CONTAINS FLOAT, "
                "\"OVERLAPS\" FLOAT, AUTHOR FLOAT, SIMDIFF FLOAT, GRSIM FLOAT, RANDDIFF FLOAT, "
                "relatedness FLOAT, {} count INT)".format("STD_DIFF FLOAT" if include_std else ""))
    for ppath in ppaths:
        print(ppath)
        # AGGREGATE reciprocal data
        select_cols = ", ".join(['target', 'candidate'] + [f"AVG(\"{x.lower()}\") as \"{x}\"" for x in judgment_labels+['relatedness']])
        if include_std:
            select_cols += ", " + "+".join([f"STDDEV_POP(\"{x.lower()}\")" for x in judgment_labels]) + 'as STD_DIFF'
        q = "INSERT INTO clean_predictions SELECT " + select_cols + ", COUNT(1) AS count FROM parquet_scan('"+ppath+"') GROUP BY target, candidate HAVING count > 1"
        con.execute(q)

CREATE VIEW clean_predictions_view AS SELECT target, candidate, AVG("swsm") as "SWSM", AVG("swde") as "SWDE", AVG("wp_dv") as "WP_DV", AVG("partof") as "PARTOF", AVG("contains") as "CONTAINS", AVG("overlaps") as "OVERLAPS", AVG("author") as "AUTHOR", AVG("simdiff") as "SIMDIFF", AVG("grsim") as "GRSIM", AVG("randdiff") as "RANDDIFF", AVG("relatedness") as "relatedness", COUNT(1) AS count FROM predictions GROUP BY target, candidate HAVING count > 1
CPU times: user 112 µs, sys: 23 µs, total: 135 µs
Wall time: 98 µs


In [46]:
# This was from a testing run, which ingested 10/48 files
con.execute('SELECT COUNT(*) FROM clean_predictions').fetch_df()

,count_star()
0,1978938


In [52]:
con.execute('SELECT COUNT(DISTINCT target) FROM clean_predictions LIMIT 10').fetch_df()

,count(target)
0,241936


In [49]:
con.execute('SELECT * FROM clean_predictions LIMIT 10').fetch_df()

,target,candidate,swsm,swde,wp_dv,partof,contains,OVERLAPS,author,simdiff,grsim,randdiff,relatedness,count
0,aeu.ark:/13960/t0rr2fz1c,t41r7dp49,0.007643,0.009654,0.325372,0.000514,0.045886,0.007719,0.561798,0.039855,0.000059,0.001500,0.089436,2
1,aeu.ark:/13960/t0rr2kz9w,t4dn4q45k,0.023531,0.006534,0.105240,0.004364,0.004411,0.027385,0.823689,0.003851,0.000922,0.000073,0.074266,2
2,aeu.ark:/13960/t0rr2m355,t3zs3h20g,0.008265,0.019025,0.819336,0.003495,0.000051,0.000056,0.037108,0.112366,0.000297,0.000002,0.123953,2
3,aeu.ark:/13960/t0rr2n76v,t01z4tv1w,0.002262,0.000198,0.004250,0.000776,0.209902,0.002839,0.775687,0.002794,0.000367,0.000924,0.073324,2
4,aeu.ark:/13960/t0rr2n76v,t44q8h11j,0.002504,0.000305,0.005673,0.000472,0.107294,0.001643,0.874128,0.004197,0.000847,0.002937,0.067066,2
5,aeu.ark:/13960/t0rr2pf8x,t86h54q00,0.000853,0.000137,0.009035,0.001390,0.000100,0.000054,0.886771,0.001089,0.003360,0.097212,0.055137,2
6,aeu.ark:/13960/t0rr2pr3j,t3kw6678m,0.013650,0.004063,0.069576,0.003168,0.003047,0.022015,0.878640,0.004451,0.001055,0.000335,0.069319,2
7,aeu.ark:/13960/t0rr2qs4w,t6rz0dr6n,0.013639,0.014038,0.015231,0.019002,0.034690,0.031277,0.870604,0.001332,0.000046,0.000142,0.069288,2
8,aeu.ark:/13960/t0rr2rh18,t48p6bx8n,0.001680,0.000374,0.008178,0.000199,0.000150,0.000064,0.866911,0.013106,0.000201,0.109136,0.053876,2
9,aeu.ark:/13960/t0rr2t345,t44q8z91h,0.005697,0.004508,0.113042,0.000072,0.000072,0.000063,0.528715,0.343709,0.000783,0.003339,0.056281,2


In [47]:
con.execute('SELECT COUNT(*) FROM predictions').fetch_df()

,count_star()
0,159528369


## Profiling

Original predictions table, not indexed or sorted:

In [4]:
%%time
df = con.execute(f"SELECT * FROM predictions WHERE target == 'pst.000022329394' OR candidate == 'pst.000022329394'").fetch_df()
df.shape

CPU times: user 14.3 s, sys: 17.3 s, total: 31.6 s
Wall time: 31.2 s


(25, 15)

Original predictions table, indexed but not sorted:

In [9]:
%%time
df = con.execute(f"SELECT * FROM predictions WHERE target == 'pst.000022329394' OR candidate == 'pst.000022329394'").fetch_df()
df.shape

CPU times: user 11.4 s, sys: 6.57 s, total: 18 s
Wall time: 17.7 s


(25, 15)

TODO: predictions sorted

## Queries

### Count all records

In [10]:
c = con.execute(f"SELECT COUNT(1) FROM predictions")
c.fetchone()

(159528369,)

In [15]:
k, v = con.execute(f"EXPLAIN TABLE predictions").fetchone()
print(v)

┌───────────────────────────┐
│          SEQ_SCAN         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        predictions        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           target          │
│         candidate         │
│            swsm           │
│            swde           │
│           wp_dv           │
│           partof          │
│          contains         │
│          OVERLAPS         │
│           author          │
│          simdiff          │
│           grsim           │
│          randdiff         │
│           guess           │
│        relatedness        │
│          flipped          │
└───────────────────────────┘                             



### See most-represented books

Hint: it's the buggy Monthly labour reviews

Count volumes that show up most often. Also group by title (e.g. 100 copies of Hamlet plus 200 copies of Hamlet).

In [ ]:
%%time
# BTW - tested performance on the view creation and joined select with and w/o index, on 500k prediction files. Was 18.3s vs 35.6s.
con.execute("DROP VIEW IF EXISTS top_targets")
con.execute("CREATE VIEW top_targets AS SELECT predictions.target, count(1) as count FROM predictions GROUP BY predictions.target ORDER BY count DESC LIMIT 10000")
df = con.execute(f"SELECT top_targets.*, meta.title, meta.description FROM top_targets JOIN meta ON (target=htid)").fetch_df()
df.head(15)

In [18]:
%%time
con.execute("DROP VIEW IF EXISTS top_candidates")
con.execute("CREATE VIEW top_candidates AS SELECT predictions.candidate, count(1) as count FROM predictions GROUP BY predictions.candidate ORDER BY count DESC LIMIT 10000")
df = con.execute(f"SELECT top_candidates.*, meta.title, meta.description FROM top_candidates JOIN meta ON (candidate=htid)").fetch_df()
df.head(25)

CPU times: user 38.7 s, sys: 2.52 s, total: 41.2 s
Wall time: 41.3 s


,candidate,count,title,description
0,umn.31951d022083041,3074,The Northwestern reporter.,v.117/118 (1908)
1,uva.x000012536,2195,The complete works of William Shakespeare / wi...,NaN
2,pst.000051385477,2158,"The complete works of William Shakespeare, edi...",NaN
3,uva.x000027677,1824,The complete works of William Shakespeare / ed...,NaN
4,umn.31951t003762797,1708,Environmental effects of producing electric po...,PT.1-2
5,uc1.31822031977861,1532,ACS surgery: principles & practice/ American C...,2003
6,uc1.b3473597,1450,A Catholic commentary on Holy Scripture. Edito...,NaN
7,osu.32435022740781,1447,"Encyclopedia Britannica; a dictionary of arts,...",v.25-26
8,umn.31951d036034709,1426,Special education and rehabilitation. Hearings...,pt.1-7
9,wu.89002222198,1421,The works of William Shakespeare / edited by H...,3


In [19]:
df.groupby('title')[['count']].sum().sort_values(['count'], ascending=False).head(25)

,count
title,
"Monthly labor review / U.S. Department of Labor, Bureau of Labor Statistics.",152927
United States Congressional serial set.,151053
The Journal of chemical physics.,107726
Science,77461
Biochimica et biophysica acta,67475
United States congressional serial set.,63850
The Pacific reporter.,54446
Bulletin - United States Geological Survey,53615
The Federal reporter : with key-number annotations ... Permanent ed. ..,48783


In [21]:
df.groupby('title')[['count']].sum().sort_values(['count'], ascending=False).head(25)

,count
title,
Hearings,310218
United States congressional serial set.,269310
The Journal of chemical physics.,205570
Annual report.,191781
Report.,183028
Soil survey.,182095
Bulletin.,145063
Biochimica et biophysica acta,135473
Proceedings.,119271


In [282]:
df.sort_values('std_diff', ascending=False)

,target,candidate,SWSM,SWDE,WP_DV,PARTOF,CONTAINS,OVERLAPS,AUTHOR,SIMDIFF,GRSIM,RANDDIFF,relatedness,std_diff,count
108803,mdp.39015082283469,mdp.39015086652388,1.453574e-10,4.706068e-09,3.192219e-05,2.697137e-14,5.000000e-01,5.376347e-12,1.324393e-09,4.999681e-01,7.698918e-16,3.842009e-18,0.070004,1.000000e+00,2
217359,mdp.39015059528649,pst.000073279570,8.025006e-11,6.116357e-13,5.000000e-01,1.858541e-11,5.000000e-01,5.661509e-16,7.663408e-09,1.419116e-10,2.782090e-17,6.843832e-19,0.130000,1.000000e+00,2
137611,mdp.39015040405444,uc1.32106008628874,5.000000e-01,4.240366e-18,5.000000e-01,6.065067e-31,5.311659e-32,1.303394e-29,9.430933e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.170000,1.000000e+00,2
25271,msu.31293006840650,umn.31951001403188i,5.000000e-01,3.819724e-30,5.000000e-01,0.000000e+00,3.757252e-36,0.000000e+00,1.291170e-31,0.000000e+00,0.000000e+00,0.000000e+00,0.170000,1.000000e+00,2
25267,msu.31293006840650,uc1.c028600471,5.000000e-01,0.000000e+00,5.000000e-01,0.000000e+00,2.166088e-34,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.170000,1.000000e+00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247129,uc1.b3074685,uva.x001725180,1.000000e+00,0.000000e+00,5.899416e-26,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.200000,5.899416e-26,2
222104,umn.31951001403188i,umn.31951001403192r,1.000000e+00,1.349578e-38,1.556127e-26,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.200000,1.542906e-26,2
248985,uc1.b3074685,uva.x001724886,1.000000e+00,1.250860e-35,1.138501e-27,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.200000,1.138498e-27,2
238562,uc1.b3074685,uc1.b3289256,1.000000e+00,0.000000e+00,5.375050e-32,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.200000,5.375050e-32,2


In [208]:
# JOINING metadata
join_syntax = "JOIN meta ON (target=meta.htid) JOIN meta AS meta2 ON (candidate=meta2.htid)"
join_select = "meta.title as target_title, meta.description as target_description, meta2.title as candidate_title, meta2.description as candidate_description"
a = con.execute(f"SELECT predictions.guess, {join_select} FROM predictions {join_syntax}")
a.fetch_df()

,guess,target_title,target_description,candidate_title,candidate_description
0,AUTHOR,Sociology and rural life,no.16,Economic opportunities in Mississippi's pine l...,NaN
1,AUTHOR,Sociology and rural life,no.30,Economic opportunities in Mississippi's pine l...,NaN
2,WP_DV,Report.,no.65-80inc.,Monthly labor review / U.S. Department of Labo...,v.49
3,SIMDIFF,Report.,no.96-110inc.,Bulletin of the United States Bureau of Labor ...,no. 699-714
4,WP_DV,Report.,no.126-140,Monthly labor review / U.S. Department of Labo...,v.55 1942 Jul-Dec
...,...,...,...,...,...
2274081,WP_DV,NaN,NaN,Publications of the Pipe roll society.,v.56 (1940)
2274082,WP_DV,NaN,NaN,Inquisitions and assessments relating to feuda...,v.2
2274083,WP_DV,NaN,NaN,Issues of the Exchequer : being payments made ...,NaN
2274084,WP_DV,NaN,NaN,Parochial antiquities attempted in the history...,v.2


In [117]:
!ls -lh /data/saddl/meta.parquet/*parquet | wc -l
!ls -lh /data/saddl/full.duckdb
!ls -lh $target

89
-rw-r--r--. 1 21901039 345200513 12K Jul  5 15:17 /data/saddl/full.duckdb
-rw-r--r--. 1 21901039 345200513 21K Jun 28 20:26 /data/saddl/full/pst/0071/pst.000072787106.predictions.parquet
